In [1]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16,ResNet50,DenseNet201,ResNet152V2,InceptionV3
from tensorflow.keras.layers import AveragePooling2D,BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import shutil
import cv2
import os
import tensorflow as tf
random_value=42 #setting randomness at fixed for every run
import logging, os

print("------loading  data-----")
data = np.load("data1.npy")
labels = np.load("label_precess.npy")
print("Total images = {} and labels = {}".format(len(data),len(labels)))
(trainX1, testX1, trainY1, testY1) = train_test_split(data, labels, test_size=0.2,stratify=labels, random_state=random_value)

#(trainX1, testX1, trainY1, testY1) = train_test_split(data, labels, test_size=0.20,stratify=labels, random_state=random_value)
# initialize the training data augmentation object
print("Size of train image : {} and train label : {}".format(len(trainX1),len(trainY1)))
print("Size of test image : {} and test label : {}".format(len(testX1),len(testY1)))

trainX = tf.convert_to_tensor(trainX1)
testX = tf.convert_to_tensor(testX1)
trainY = tf.convert_to_tensor(trainY1)
testY = tf.convert_to_tensor(testY1)

from tensorflow import keras
METRICS = ["accuracy",
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy2'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

2022-05-19 23:27:17.399039: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


------loading  data-----
Total images = 4500 and labels = 4500
Size of train image : 3600 and train label : 3600
Size of test image : 900 and test label : 900


2022-05-19 23:27:22.430970: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-05-19 23:27:22.594939: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA A40 computeCapability: 8.6
coreClock: 1.74GHz coreCount: 84 deviceMemorySize: 44.37GiB deviceMemoryBandwidth: 648.29GiB/s
2022-05-19 23:27:22.594968: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-05-19 23:27:22.598117: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-05-19 23:27:22.598150: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-05-19 23:27:22.598799: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.so.10
2022

In [15]:
baseModel3 = DenseNet201(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)),)
headModel3 = baseModel3.output
# headModel = GaussianNoise(0.9)(headModel)
headModel3 = AveragePooling2D(pool_size=(4, 4))(headModel3)
headModel3 = Flatten(name="flatten")(headModel3)
headModel3 = BatchNormalization()(headModel3)
headModel3 = Dense(64, activation="relu")(headModel3)
headModel3 = Dropout(0.5)(headModel3)
headModel3 = Dense(3, activation="softmax")(headModel3)
model3= Model(inputs=baseModel3.input, outputs=headModel3)
for layer in baseModel3.layers:
    layer.trainable =False


BS_den = 64
EPOCHS_den = 100
INIT_LR_den = 5e-4
num_den = 15 #核心数量
print("[INFO] compiling dense model...")
opt3 = Adam(lr=INIT_LR_den, decay=INIT_LR_den / EPOCHS_den)
model3.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt3, metrics=METRICS)
print("[INFO] training resnet model...")
    

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("freeze_dens.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto',restore_best_weights=True)
hist=model3.fit(trainX, trainY, batch_size=64, epochs=100, verbose=1, validation_split=0.1, callbacks=[checkpoint,early], workers=15, use_multiprocessing= True)
    
model3.save('freeze_dens.h5')
print("[INFO] densenet model saved...")

[INFO] compiling dense model...
[INFO] training resnet model...


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/100
51/51 [==============================] - 21s 205ms/step - loss: 0.6174 - accuracy: 0.7503 - tp: 2645.0000 - fp: 675.0000 - tn: 6525.0000 - fn: 955.0000 - accuracy2: 0.8491 - precision: 0.7967 - recall: 0.7347 - auc: 0.9151 - val_loss: 0.4300 - val_accuracy: 0.8306 - val_tp: 289.0000 - val_fp: 44.0000 - val_tn: 676.0000 - val_fn: 71.0000 - val_accuracy2: 0.8935 - val_precision: 0.8679 - val_recall: 0.8028 - val_auc: 0.9563

Epoch 00001: val_accuracy improved from -inf to 0.83056, saving model to freeze_dens.h5
Epoch 2/100
51/51 [==============================] - 7s 146ms/step - loss: 0.3017 - accuracy: 0.8852 - tp: 2840.0000 - fp: 335.0000 - tn: 6145.0000 - fn: 400.0000 - accuracy2: 0.9244 - precision: 0.8945 - recall: 0.8765 - auc: 0.9741 - val_loss: 0.3282 - val_accuracy: 0.9000 - val_tp: 318.0000 - val_fp: 32.0000 - val_tn: 688.0000 - val_fn: 42.0000 - val_accuracy2: 0.9315 - val_precision: 0.9086 - val_recall: 0.8833 - val_auc: 0.9774

Epoch 00002: val_accuracy improved 

In [16]:
baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)),)
headModel = baseModel.output
# headModel = GaussianNoise(0.9)(headModel)
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = BatchNormalization()(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
    layer.trainable = False


BS = 64
EPOCHS = 100
INIT_LR = 5e-4
num = 15 #核心数量
print("[INFO] compiling vgg16 model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# model.compile(loss="binary_crossentropy", optimizer=opt, metrics=METRICS)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=METRICS)
print("[INFO] training head...")


from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("freeze_VGG16.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto',restore_best_weights=True)
hist=model.fit(trainX, trainY, batch_size=64, epochs=100, verbose=1, validation_split=0.1,callbacks=[checkpoint,early], workers=15, use_multiprocessing= True)

model.save('freeze_VGG16.h5')
print("[INFO] vgg16 model saved...")


[INFO] compiling vgg16 model...
[INFO] training head...
Epoch 1/100
51/51 [==============================] - 14s 146ms/step - loss: 0.8676 - accuracy: 0.6136 - tp: 2094.0000 - fp: 962.0000 - tn: 6238.0000 - fn: 1506.0000 - accuracy2: 0.7715 - precision: 0.6852 - recall: 0.5817 - auc: 0.8286 - val_loss: 0.9550 - val_accuracy: 0.6917 - val_tp: 6.0000 - val_fp: 0.0000e+00 - val_tn: 720.0000 - val_fn: 354.0000 - val_accuracy2: 0.6722 - val_precision: 1.0000 - val_recall: 0.0167 - val_auc: 0.8680

Epoch 00001: val_accuracy improved from -inf to 0.69167, saving model to freeze_VGG16.h5
Epoch 2/100
51/51 [==============================] - 4s 88ms/step - loss: 0.4899 - accuracy: 0.8015 - tp: 2477.0000 - fp: 538.0000 - tn: 5942.0000 - fn: 763.0000 - accuracy2: 0.8662 - precision: 0.8216 - recall: 0.7645 - auc: 0.9347 - val_loss: 0.8547 - val_accuracy: 0.7778 - val_tp: 72.0000 - val_fp: 1.0000 - val_tn: 719.0000 - val_fn: 288.0000 - val_accuracy2: 0.7324 - val_precision: 0.9863 - val_recall: 0.2

In [35]:
baseModel4 = InceptionV3(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)),)
headModel4 = baseModel4.output
# headModel = GaussianNoise(0.9)(headModel)
headModel4 = AveragePooling2D(pool_size=(4, 4))(headModel4)
headModel4 = Flatten(name="flatten")(headModel4)
headModel4 = BatchNormalization()(headModel4)
headModel4 = Dense(64, activation="relu")(headModel4)
headModel4 = Dropout(0.5)(headModel4)
headModel4 = Dense(3, activation="softmax")(headModel4)
model4= Model(inputs=baseModel4.input, outputs=headModel4)
for layer in baseModel4.layers:
    layer.trainable = False

BS_den1 = 64
EPOCHS_den1 = 100
INIT_LR_den1 = 5e-4
num_den1 = 15 #核心数量
print("[INFO] compiling inv3 model...")
opt4 = Adam(lr=INIT_LR_den1, decay=INIT_LR_den1 / EPOCHS_den1)
model4.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=METRICS)
print("[INFO] training inv3 model...")

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint_iv = ModelCheckpoint("freeze_incv3.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
early_iv = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto',restore_best_weights=True)
hist=model4.fit(trainX, trainY, batch_size=64, epochs=100, verbose=1, validation_split=0.1, callbacks=[checkpoint_iv,early_iv], workers=15, use_multiprocessing= True)
model4.save('freeze_incv31.h5')
print("[INFO] vgg16 model saved...")

[INFO] compiling inv3 model...
[INFO] training inv3 model...
Epoch 1/100
51/51 [==============================] - 14s 178ms/step - loss: 0.6706 - accuracy: 0.7938 - tp: 3056.0000 - fp: 808.0000 - tn: 7032.0000 - fn: 864.0000 - accuracy2: 0.8578 - precision: 0.7909 - recall: 0.7796 - auc: 0.9260 - val_loss: 0.5598 - val_accuracy: 0.7861 - val_tp: 273.0000 - val_fp: 70.0000 - val_tn: 650.0000 - val_fn: 87.0000 - val_accuracy2: 0.8546 - val_precision: 0.7959 - val_recall: 0.7583 - val_auc: 0.9248

Epoch 00001: val_accuracy improved from -inf to 0.78611, saving model to freeze_incv3.h5
Epoch 2/100
51/51 [==============================] - 3s 59ms/step - loss: 0.2886 - accuracy: 0.8855 - tp: 2864.0000 - fp: 365.0000 - tn: 6115.0000 - fn: 376.0000 - accuracy2: 0.9238 - precision: 0.8870 - recall: 0.8840 - auc: 0.9766 - val_loss: 0.4010 - val_accuracy: 0.8278 - val_tp: 295.0000 - val_fp: 57.0000 - val_tn: 663.0000 - val_fn: 65.0000 - val_accuracy2: 0.8870 - val_precision: 0.8381 - val_recall: 

In [38]:
model_iv=tf.keras.models.load_model('./freeze_incv31.h5')
for layer in model_den.layers:
    layer.trainable = True
opt4 = Adam(lr=1e-5, decay=1e-5 / 20)
model_iv.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt4, metrics=METRICS)
model_iv.fit(trainX, trainY, batch_size=64, epochs=20, verbose=1, validation_split=0.1, callbacks=[checkpoint_iv,early_iv], workers=15, use_multiprocessing= True)

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/20
51/51 [==============================] - 10s 120ms/step - loss: 0.0902 - accuracy: 0.9685 - tp: 3461.0000 - fp: 136.0000 - tn: 7064.0000 - fn: 139.0000 - accuracy2: 0.9745 - precision: 0.9622 - recall: 0.9614 - auc: 0.9958 - val_loss: 0.3091 - val_accuracy: 0.9111 - val_tp: 328.0000 - val_fp: 32.0000 - val_tn: 688.0000 - val_fn: 32.0000 - val_accuracy2: 0.9407 - val_precision: 0.9111 - val_recall: 0.9111 - val_auc: 0.9764

Epoch 00001: val_accuracy did not improve from 0.91667
Epoch 2/20
51/51 [==============================] - 3s 61ms/step - loss: 0.0812 - accuracy: 0.9694 - tp: 3137.0000 - fp: 98.0000 - tn: 6382.0000 - fn: 103.0000 - accuracy2: 0.9793 - precision: 0.9697 - recall: 0.9682 - auc: 0.9983 - val_loss: 0.3112 - val_accuracy: 0.9111 - val_tp: 328.0000 - val_fp: 32.0000 - val_tn: 688.0000 - val_fn: 32.0000 - val_accuracy2: 0.9407 - val_precision: 0.9111 - val_recall: 0.9111 - val_auc: 0.9761

Epoch 00002: val_accuracy did not improve from 0.91667
Epoch 3/20
51/51 

In [39]:
from sklearn import metrics
print(metrics.accuracy_score(np.argmax(model4.predict(testX),1),np.argmax(testY,1)))
metrics.accuracy_score(np.argmax(model_iv.predict(testX),1),np.argmax(testY,1))

0.9022222222222223


0.9022222222222223

In [40]:
model_VGG.save('fine-tune-iv3.h5')